In [1]:
import os
import json
import pandas as pd
import traceback

In [9]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

In [10]:
KEY=os.getenv("OPENAI_API_KEY")

In [12]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)